<a href="https://colab.research.google.com/github/mon3/deepNN_NER/blob/master/elmo_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q pydrive

    100% |████████████████████████████████| 993kB 21.8MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
your_module = drive.CreateFile({'id':'1lBy0_BsIQ_gzuCkM-P4MWbstIZFkHj8L'})
your_module.GetContentFile('preprocess.py')


In [0]:
prepro_elmo_module = drive.CreateFile({'id':'1PV_OV65fPWh82wfuoabTKNdCEMIhkAWN'})
prepro_elmo_module.GetContentFile('prepro_elmo.py')

In [0]:
validation_module = drive.CreateFile({'id': '1nEox1MVwcpP3Fu5RGJzhKi358x9jTA2h'})
validation_module.GetContentFile('validation.py')

In [0]:
"""Load packages"""

import matplotlib.pyplot as plt
import numpy as np
import os

from validation import compute_f1
from keras.models import Model, load_model
from keras.layers import TimeDistributed, Conv1D, Dense, Embedding, Input, Dropout, LSTM, Bidirectional, MaxPooling1D, \
    Flatten, concatenate, Lambda
from preprocess import readfile, createBatches, iterate_minibatches, addCharInformation, padding
from prepro_elmo import createMatrices
from keras.utils import plot_model
from keras.initializers import RandomUniform
from keras.optimizers import SGD, Nadam
import tensorflow_hub as hub
import tensorflow as tf


In [0]:
def data_prep_ELMo(train_x, train_y, test_x, test_y, max_len):
    INDEX_FROM = 3
    word_to_index = imdb.get_word_index()
    word_to_index = {k: (v + INDEX_FROM) for k, v in word_to_index.items()}

    word_to_index["<START>"] = 1
    word_to_index["<UNK>"] = 2

    index_to_word = {v: k for k, v in word_to_index.items()}

    sentences = []
    for i in range(len(train_x)):
        temp = [index_to_word[ids] for ids in train_x[i]]
        sentences.append(temp)

    test_sentences = []
    for i in range(len(test_x)):
        temp = [index_to_word[ids] for ids in test_x[i]]
        test_sentences.append(temp)

    train_text = [' '.join(sentences[i][:max_len]) for i in range(len(sentences))]
    train_text = np.array(train_text, dtype=object)[:, np.newaxis]
    train_label = train_y.tolist()

    test_text = [' '.join(test_sentences[i][:500]) for i in range(len(test_sentences))]
    test_text = np.array(test_text, dtype=object)[:, np.newaxis]
    test_label = test_y.tolist()

    return train_text, train_label, test_text, test_label


In [10]:
"""Set parameters"""

EPOCHS = 80               # paper: 80
DROPOUT = 0.68           # paper: 0.68
DROPOUT_RECURRENT = 0.0  # not specified in paper, 0.25 recommended; in other papers: 0.0
LSTM_STATE_SIZE = 275    # paper: 275
CONV_SIZE = 3             # paper: 3
LEARNING_RATE = 0.0105    # paper 0.0105
OPTIMIZER = Nadam()       # paper uses SGD(lr=self.learning_rate), Nadam() recommended
LOCATION_POINTER = '/content/gdrive/My Drive/masters_thesis/'
CHAR_EMBEDDING = 30   # paper: 25, previously: 30
VERSION = 10
EMBEDDING = "elmo"

Instructions for updating:
Colocations handled automatically by placer.


W0323 12:30:42.635641 140082241873792 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [29]:
"""Initialise class"""

class CNN_BLSTM(object):
    
    def __init__(self, EPOCHS, DROPOUT, DROPOUT_RECURRENT, LSTM_STATE_SIZE, CONV_SIZE, LEARNING_RATE, OPTIMIZER, CHAR_EMBEDDING, VERSION, EMBEDDING):
        
        self.epochs = EPOCHS
        self.dropout = DROPOUT
        self.dropout_recurrent = DROPOUT_RECURRENT
        self.lstm_state_size = LSTM_STATE_SIZE
        self.conv_size = CONV_SIZE
        self.learning_rate = LEARNING_RATE
        self.optimizer = OPTIMIZER
        self.char_embedding_size = CHAR_EMBEDDING
        self.version = VERSION
        self.embedding = EMBEDDING
        
    def loadData(self):
        """Load data and add character information"""
        self.trainSentences = readfile(os.path.join(LOCATION_POINTER, "data/train.txt"))
        self.devSentences = readfile(os.path.join(LOCATION_POINTER, "data/dev.txt"))
        self.testSentences = readfile(os.path.join(LOCATION_POINTER, "data/test.txt"))

    def addCharInfo(self):
        # format: [['EU', ['E', 'U'], 'B-ORG\n'], ...]
        self.trainSentences = addCharInformation(self.trainSentences)
        self.devSentences = addCharInformation(self.devSentences)
        self.testSentences = addCharInformation(self.testSentences)

    def embed(self):
        """Create word- and character-level embeddings"""

        labelSet = set()
        words = {}

        # unique words and labels in data  
        for dataset in [self.trainSentences, self.devSentences, self.testSentences]:
            for sentence in dataset:
                for token, char, label in sentence:
                    # token ... token, char ... list of chars, label ... BIO labels   
                    labelSet.add(label)
                    words[token.lower()] = True
                    
        # mapping for labels
        self.label2Idx = {}
        for label in labelSet:
            self.label2Idx[label] = len(self.label2Idx)

        # mapping for token cases
        case2Idx = {'numeric': 0, 'allLower': 1, 'allUpper': 2, 'initialUpper': 3, 'other': 4, 'mainly_numeric': 5,
                    'contains_digit': 6, 'PADDING_TOKEN': 7}
        # creates identity matrix for token cases
        self.caseEmbeddings = np.identity(len(case2Idx), dtype='float32')  # identity matrix used 

        # read GLoVE word embeddings
        word2Idx = {}
#         self.wordEmbeddings = []
        
#         word represented as 50-dim vector
#         ToDO: test with 300-dim vectors (GloVE 42B, GloVE 84B)
#         if self.embedding == "fastText":
#           fEmbeddings = open(os.path.join(LOCATION_POINTER, "embeddings/wiki-news-300d-1M.vec"), encoding="utf-8")
#         else:
#           fEmbeddings = open(os.path.join(LOCATION_POINTER, "embeddings/glove.6B.50d.txt"), encoding="utf-8")

#         # loop through each word in embeddings
#         for i, line in enumerate(fEmbeddings):
#             if i==0 and self.embedding == "fastText":
#                 continue
                
#             split = line.strip().split(" ") # removes leading and trailing chars and splits into list of single values
#             word = split[0]  # embedding word entry

#             if len(word2Idx) == 0:  # add padding+unknown
#                 word2Idx["PADDING_TOKEN"] = len(word2Idx)
#                 vector = np.zeros(len(split) - 1)  # zero vector for 'PADDING' word
#                 self.wordEmbeddings.append(vector)

#                 word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
#                 vector = np.random.uniform(-0.25, 0.25, len(split) - 1)  # zero vector for 'PADDING' word
#                 self.wordEmbeddings.append(vector)

#             if split[0].lower() in words:
#                 vector = np.array([float(num) for num in split[1:]])
#                 self.wordEmbeddings.append(vector)  # word embedding vector
#                 word2Idx[split[0]] = len(word2Idx)  # corresponding word dict; increments by 1 for each word

#         self.wordEmbeddings = np.array(self.wordEmbeddings)

        # dictionary of all possible characters
        self.char2Idx = {"PADDING": 0, "UNKNOWN": 1}
        for c in " 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,-_()[]{}!?:;#'\"/\\%$`&=*+@^~|<>":
            self.char2Idx[c] = len(self.char2Idx) # 2,3,4 ...

        # format: [[wordindices], [caseindices], [padded word indices], [label indices]]
        self.train_set = padding(createMatrices(self.trainSentences, self.label2Idx, case2Idx, self.char2Idx))
        self.dev_set = padding(createMatrices(self.devSentences, self.label2Idx, case2Idx, self.char2Idx))
        self.test_set = padding(createMatrices(self.testSentences, self.label2Idx, case2Idx, self.char2Idx))

        self.idx2Label = {v: k for k, v in self.label2Idx.items()}  # index to label(reverted)
                                                                
                                                                
    def createBatches(self):
        
        """Create batches"""
        self.train_batch, self.train_batch_len = createBatches(self.train_set)
        self.dev_batch, self.dev_batch_len = createBatches(self.dev_set)
        self.test_batch, self.test_batch_len = createBatches(self.test_set)
        
    def tag_dataset(self, dataset, model):
        """Tag data with numerical values"""
        correctLabels = []
        predLabels = []
        for i, data in enumerate(dataset):
            tokens, casing, char, labels = data
            tokens = np.asarray([tokens])
            casing = np.asarray([casing])
            char = np.asarray([char])
            pred = model.predict([tokens, casing, char], verbose=False)[0]
            pred = pred.argmax(axis=-1)  # Predict the classes
            correctLabels.append(labels)
            predLabels.append(pred)
        return predLabels, correctLabels
    
    

    def ELMoEmbedding(self, x):
        """ Return elmo embedding using tf_hub """
        elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
        # signature points to the purpose of why we would like to use the modules
        return elmo_model(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["word_emb"]
   
    
    def buildModel(self):
        """Model layers"""

        # character input
        character_input = Input(shape=(None, 52,), name="Character_input")  #input N sentences, each 52 chras length
        embed_char_out = TimeDistributed(
            Embedding(len(self.char2Idx),self.char_embedding_size, embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name="Character_embedding")(
            character_input)

        dropout = Dropout(self.dropout)(embed_char_out)

        # CNN
        conv1d_out = TimeDistributed(Conv1D(kernel_size=self.conv_size, filters=30, padding='same', activation='tanh', strides=1), name="Convolution")(dropout)
        maxpool_out = TimeDistributed(MaxPooling1D(52), name="Maxpool")(conv1d_out)  # pool_size=52: max sentence length
        char = TimeDistributed(Flatten(), name="Flatten")(maxpool_out)
        char = Dropout(self.dropout)(char)

        # word-level input
        words_input = Input(shape=(None,), dtype=tf.string, name='words_input')

        words = Lambda(self.ELMoEmbedding, output_shape=(1024,))(words_input)
#         words_input = Input(shape=(None,), dtype='int32', name='words_input')
#         words = Embedding(input_dim=self.wordEmbeddings.shape[0], output_dim=self.wordEmbeddings.shape[1], weights=[self .wordEmbeddings],
#                           trainable=False)(words_input)  # trainable=False since we provide word embeddings

        # case-info input
        casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
        casing = Embedding(output_dim=self.caseEmbeddings.shape[1], input_dim=self.caseEmbeddings.shape[0], weights=[self.caseEmbeddings],
                           trainable=False)(casing_input)  ## trainable=False since we provide case embeddings
      
#         print("SIZES FOR CONCATENATION: {} {} {}".format(casing.output_dim, words.output_dim, embed_char_out.output_dim))
        # concat & BLSTM
        output = concatenate([words, casing, char])
        output = Bidirectional(LSTM(self.lstm_state_size, 
                                    return_sequences=True, 
                                    dropout=self.dropout,                        # on input to each LSTM block
                                    recurrent_dropout=self.dropout_recurrent     # on recurrent input signal
                                   ), name="BLSTM")(output)
        output = TimeDistributed(Dense(len(self.label2Idx), activation='softmax'),name="Softmax_layer")(output)

        # set up model
        self.model = Model(inputs=[words_input, casing_input, character_input], outputs=[output])
        
        self.model.compile(loss='sparse_categorical_crossentropy', optimizer=self.optimizer)
        
        self.init_weights = self.model.get_weights()
        
        plot_model(self.model, to_file=os.path.join(LOCATION_POINTER, 'model_{}.png'.format(self.version)))
        
        print("Model built. Saved model.png\n")
        
    def train(self):
        """Default training"""

        self.f1_test_history = []
        self.f1_dev_history = []

        for epoch in range(self.epochs):    
            print("Epoch {}/{}".format(epoch, self.epochs))
            # batch: [word_indices, case_indices, char_indices]
            for i,batch in enumerate(iterate_minibatches(self.train_batch,self.train_batch_len)):
                labels, tokens, casing,char = batch       
                self.model.train_on_batch([tokens, casing,char], labels)

            # compute F1 scores
            predLabels, correctLabels = self.tag_dataset(self.test_batch, self.model)
            pre_test, rec_test, f1_test = compute_f1(predLabels, correctLabels, self.idx2Label)
            self.f1_test_history.append(f1_test)
            print("f1 test ", round(f1_test, 4))

            predLabels, correctLabels = self.tag_dataset(self.dev_batch, self.model)
            pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, self.idx2Label)
            self.f1_dev_history.append(f1_dev)
            print("f1 dev ", round(f1_dev, 4), "\n")
            
        print("Final F1 test score: ", f1_test)
            
        print("Training finished.")
            
        # save model
        self.modelName = "{}_{}_{}_{}_{}_{}_{}_{}_{}_{}".format(self.epochs, 
                                                        self.dropout, 
                                                        self.dropout_recurrent, 
                                                        self.lstm_state_size,
                                                        self.conv_size,
                                                        self.learning_rate,
                                                        self.char_embedding_size,
                                                        self.optimizer.__class__.__name__,
                                                        self.version,
                                                        self.embedding
                                                       )
        
        modelName = self.modelName + ".h5"
        self.model.save(os.path.join(LOCATION_POINTER, modelName))
        print("Model weights saved.")
        
        self.model.set_weights(self.init_weights)  # clear model
        print("Model weights cleared.")

    def writeToFile(self):
        """Write output to file"""

        output = np.matrix([[int(i) for i in range(self.epochs)], self.f1_test_history, self.f1_dev_history])

        fileName = os.path.join(LOCATION_POINTER, self.modelName + ".txt")
        with open(fileName,'wb') as f:
            for line in output:
                np.savetxt(f, line, fmt='%.5f')
                
        print("Model performance written to file.")
        
    def saveResults(self):
        plt.plot(cnn_blstm.f1_test_history, label = "F1 test")
        plt.plot(cnn_blstm.f1_dev_history, label = "F1 dev")
        plt.xlabel("Epochs")
        plt.ylabel("F1 score")
        plt.legend()
        plt.savefig(os.path.join(LOCATION_POINTER, self.modelName + ".png"))

    print("Class initialised.")

Class initialised.


In [28]:
"""Construct and run model"""

cnn_blstm = CNN_BLSTM(EPOCHS, DROPOUT, DROPOUT_RECURRENT, LSTM_STATE_SIZE, CONV_SIZE, LEARNING_RATE, OPTIMIZER, CHAR_EMBEDDING, VERSION, EMBEDDING)
cnn_blstm.loadData()
cnn_blstm.addCharInfo()
cnn_blstm.embed()
cnn_blstm.createBatches()
cnn_blstm.buildModel()
cnn_blstm.train()
cnn_blstm.writeToFile()
cnn_blstm.saveResults()

NameError: ignored

In [3]:
from keras.datasets import imdb

def load_data(vocab_size, max_len):
    """
        Loads the keras imdb dataset

        Args:
            vocab_size = {int} the size of the vocabulary
            max_len = {int} the maximum length of input considered for padding

        Returns:
            X_train = tokenized train data
            X_test = tokenized test data

    """
    INDEX_FROM = 3

    (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size, index_from=INDEX_FROM)

    return X_train, X_test, y_train, y_test

Using TensorFlow backend.


In [4]:
    x_train,x_test,y_train,y_test = load_data(10000, 200)


17465344/17464789 [==============================] - 0s 0us/step


In [0]:
import numpy as np
def data_prep_ELMo(train_x, train_y, test_x, test_y, max_len):
    INDEX_FROM = 3
    train_text,train_label,test_text,test_label = data_prep_ELMo(x_train,y_train,x_test,y_test,200)
    word_to_index = {k: (v + INDEX_FROM) for k, v in word_to_index.items()}

    word_to_index["<START>"] = 1
    word_to_index["<UNK>"] = 2

    index_to_word = {v: k for k, v in word_to_index.items()}

    sentences = []
    for i in range(len(train_x)):
        temp = [index_to_word[ids] for ids in train_x[i]]
        sentences.append(temp)

    test_sentences = []
    for i in range(len(test_x)):
        temp = [index_to_word[ids] for ids in test_x[i]]
        test_sentences.append(temp)

        
    print("first: {}".format(' '.join(sentences[0][:max_len])))
    train_text = [' '.join(sentences[i][:max_len]) for i in range(len(sentences))]
    train_text = np.array(train_text, dtype=object)[:, np.newaxis]
    train_label = train_y.tolist()

    test_text = [' '.join(test_sentences[i][:500]) for i in range(len(test_sentences))]
    test_text = np.array(test_text, dtype=object)[:, np.newaxis]
    test_label = test_y.tolist()

    return train_text, train_label, test_text, test_label

In [26]:
train_text,train_label,test_text,test_label = data_prep_ELMo(x_train, y_train,x_test, y_test, 200)


RecursionError: ignored

In [0]:
train_text,train_label,test_text,test_label = data_prep_ELMo(x_train,y_train,x_test,y_test,200)


In [20]:
print(train_text[0])

["<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for w